# sktime Algorithms Catalog and Taxonomy

## Purpose
Build a mental map of sktime estimator families and learn how to enumerate every algorithm available in your local installation.

## What you will do here
- Understand the main time series learning tasks sktime supports.
- Learn the scitype concept and how it maps to estimator classes.
- Use the registry to list all estimators, filter by task, and visualize the catalog.


## Core time series notation

We use standard time series notation:

- Univariate series: $y_t$ for time index $t = 1, \dots, T$.
- Multivariate series: $\mathbf{y}_t \in \mathbb{R}^d$.
- Exogenous variables: $\mathbf{x}_t$ (optional).
- Forecast horizon: $h$, with predictions $\hat{y}_{T+1:T+h}$.

These conventions help you keep inputs, targets, and horizons straight across tasks.


## Learning tasks supported by sktime

sktime supports several core time series learning tasks. The exact set of tasks can evolve between versions, so use the registry below to confirm what is available in your environment.

| Task | Input | Output | Typical objective |
| --- | --- | --- | --- |
| Forecasting | Past values (and optional exogenous variables) | Future values | Predict future trajectory |
| Classification | One or more series | Class label | Assign a discrete label |
| Regression | One or more series | Continuous value | Predict a real-valued target |
| Clustering | A collection of series | Cluster assignment | Group similar series |
| Annotation | A series | Time points or segments | Outliers, anomalies, change points, segmentation |

Reference: https://www.sktime.net/en/v0.26.0/get_started.html


## Estimator scitypes and tags

In sktime, each estimator belongs to a scitype (base class) such as forecaster, classifier, regressor, clusterer, transformer, or series-annotator. The exact list is version-dependent and is exposed via the registry.

Reference: https://www.sktime.net/en/v0.32.3/api_reference/auto_generated/sktime.registry._tags.object_type.html


In [ ]:
# List estimator scitypes registered in this sktime version
try:
    from sktime.registry import BASE_CLASS_REGISTER

    print('Registered scitypes:')
    for scitype, class_ref in BASE_CLASS_REGISTER:
        print(f'- {scitype}: {class_ref}')
except Exception as exc:
    print('sktime is not installed or registry import failed:', exc)


## Dynamic catalog of algorithms

The `all_estimators` registry query lets you enumerate every estimator that ships with your installed sktime version. This is the most reliable way to capture all algorithms without maintaining a static list.

For a deeper, module-level catalog (including a treemap view), see `data_science/time_series/sktime_algorithms/registry/00_overview.ipynb`.

Reference: https://www.sktime.net/en/v0.35.0/api_reference/auto_generated/sktime.registry.all_estimators.html


In [ ]:
# Build a dataframe of all estimators with selected tags
try:
    import pandas as pd
    from sktime.registry import all_estimators

    estimators = all_estimators(as_dataframe=True, return_tags=['object_type'])
    estimators.head(10)
except Exception as exc:
    print('Install sktime (and pandas) to run this cell:', exc)


### Filter by task

Once you know the scitype you want, filter the registry to a concrete candidate set.


In [ ]:
try:
    from sktime.registry import all_estimators

    forecasters = all_estimators(estimator_types='forecaster', as_dataframe=True)
    classifiers = all_estimators(estimator_types='classifier', as_dataframe=True)
    regressors = all_estimators(estimator_types='regressor', as_dataframe=True)

    print(f'Forecasters: {len(forecasters)}')
    print(f'Classifiers: {len(classifiers)}')
    print(f'Regressors: {len(regressors)}')
except Exception as exc:
    print('Registry query failed:', exc)


### Inspect available tags

Tags describe estimator capabilities (for example, handling multivariate series or missing values). Use `all_tags` to discover valid tags for a given scitype, then filter `all_estimators` with `filter_tags`.

Reference: https://www.sktime.net/en/v0.37.0/api_reference/tags.html


In [ ]:
try:
    from sktime.registry import all_tags

    tags = all_tags(estimator_types='forecaster', as_dataframe=True)
    tags.head(10)
except Exception as exc:
    print('Tag inspection failed:', exc)


## Visual overview with Plotly

A quick count of estimators by scitype helps you see where sktime has the most coverage.


In [ ]:
try:
    import plotly.express as px

    def to_list(value):
        if value is None:
            return []
        if isinstance(value, (list, tuple, set)):
            return list(value)
        return [value]

    tmp = estimators.copy()
    tmp['object_type'] = tmp['object_type'].apply(to_list)
    exploded = tmp.explode('object_type')

    counts = exploded['object_type'].value_counts().reset_index()
    counts.columns = ['scitype', 'count']

    fig = px.bar(
        counts,
        x='scitype',
        y='count',
        title='sktime estimators by scitype'
    )
    fig.show()
except Exception as exc:
    print('Plotly chart skipped:', exc)


## Practical selection workflow

1. Identify the task (forecasting, classification, regression, clustering, annotation).
2. Filter by scitype using the registry.
3. Narrow down with tags (capabilities like multivariate support, missing values, or probabilistic outputs).
4. Benchmark a short list with task-appropriate metrics and backtesting or cross-validation.

This workflow scales better than searching for algorithms by name and keeps your choices aligned with your data constraints.
